In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from MetabolomicsPaper.GeneralHelperFunctions import *

paper_dir = '/net/mraid08/export/jafar/Microbiome/Analyses/Noamba/Cardio/ACS_study/v2/'
curr_dir = os.path.join(paper_dir, 'EGA')
make_dir_if_not_exists(curr_dir)
os.chdir(paper_dir)

In [34]:
acs_full_data = pd.read_csv(os.path.join('data', 'ACS_full_data.csv'), index_col=0)
# total number of non-ACS
pnp1_full_data = pd.read_csv(os.path.join('data', 'pnp1_full_data.csv'), index_col=1)

/usr/wisdom/python-3.7.4/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# metabolon

In [2]:
from LabData.DataLoaders.MetabolonLoader import MetabolonLoader

In [3]:
ms_data = MetabolonLoader().get_data(metabolon_runs=['Metabolon_MAY2018'])

In [30]:
acs_pnp1_anchor = ms_data.df_metadata[ms_data.df_metadata.StudyTypeID != 5].index
ms_data.df_metadata.RegistrationCode.fillna(ms_data.df_metadata.CLIENT_IDENTIFIER, inplace=True)
df = ms_data.df.loc[acs_pnp1_anchor].dropna(how='all', axis=1).copy()
df.index = ms_data.df_metadata.loc[acs_pnp1_anchor, 'RegistrationCode'].values
df.index.names = ['RegistrationCode']
df_metadata = ms_data.df_metadata.loc[acs_pnp1_anchor].set_index('RegistrationCode').copy()
df_metadata.drop(['BARCODE', 'CLIENT_IDENTIFIER', 'COMMENT', 'SAMPLE_ID', 'SAMPLE_NAME', 'metabolon_run'], axis=1, inplace=True)
df_metadata.replace({'StudyTypeID':{8: 'ACS', 1:'non-ACS'}}, inplace=True) 
metabolites = ms_data.metabolites.loc[df.columns].drop(['metabolon_run', 'metabolon_run_id'], axis=1)

df.to_csv(os.path.join(curr_dir, 'metabolon_data_may18.csv'))
df_metadata.to_csv(os.path.join(curr_dir, 'metabolon_metadata_may18.csv'))
metabolites.to_csv(os.path.join(curr_dir, 'metabolon_metabolites_may18.csv'))

# nightingale

In [31]:
from LabData.DataLoaders.NightingaleLoader import NightingaleLoader

ntg = NightingaleLoader().get_data()

In [42]:
pd.concat((pnp1_full_data, acs_full_data), axis=0).loc[:, ntg.metabolites.index].dropna(how='all', axis=0).to_csv(os.path.join(curr_dir, 'nightingale_data.csv'))
ntg.metabolites.drop(['nightingale_run', 'nightingale_run_id'], axis=1).to_csv(os.path.join(curr_dir, 'nightingale_metabolites.csv'))

# ACS clinical data

In [58]:
medical_db_path = '../data/ACS_Medical_database_17.04.2020_legend.csv'
acs_data_legend = pd.read_csv(medical_db_path, index_col=0).drop(['ID', 'Timestamp', 'Date of Birth'], axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
acs_data_legend.to_csv(os.path.join(curr_dir, 'acs_data_legend.csv'))
acs_full_data.iloc[:, :237].drop(['ID', 'Timestamp', 'Date of Birth'], axis=1).dropna(how='all', axis=0).to_csv(os.path.join(curr_dir, 'acs_data_for_EGA.csv'))

# Healthy clinical data

In [68]:
diagnosis_by_drugs = pd.read_excel(os.path.join('../data', 'pnp_drugs-diagnosis.xls'), 'Diagnosis')

In [69]:
pnp1_full_data.loc[:, 'treatment for DM'] = pnp1_full_data.reindex(diagnosis_by_drugs['DM'].dropna(), axis=1).any(1)
pnp1_full_data.loc[:, 'treatment for Hypertension'] = pnp1_full_data.reindex(diagnosis_by_drugs['Hypertension'].dropna(), axis=1).any(1)
pnp1_full_data.loc[:, 'treatment for Dyslipidemia'] = pnp1_full_data.reindex(diagnosis_by_drugs['Dyslipidemia'].dropna(), axis=1).any(1)

In [76]:
cols = ['Age', 'gender', 'bmi', 'smoke_tobacco_now', 'DM', 'pre-diabetes', 'Dyslipidemia', 'Hypertension', 'bt__hba1c', 'LDL_nightingale_friedewald', 
        'treatment for DM', 'treatment for Hypertension', 'treatment for Dyslipidemia', 'FD']
pnp1_full_data[cols].to_csv(os.path.join(curr_dir, 'pnp1_clinical_data.csv'))